In [59]:
import pandas as pd
import numpy as np
import os
from pyspark import SparkConf 
from pyspark.sql import SparkSession # https://spark.apache.org/docs/1.6.1/sql-programming-guide.html
from pyspark.sql import functions as F # access to the sql functions https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions
from IPython.display import HTML

In [32]:
# To persist our sparksql data from session to session we will use derby.
warehouse_location = os.path.abspath('../data/spark-warehouse')
java_options = "-Dderby.system.home=" + warehouse_location
print(warehouse_location)
print(java_options)
# make sure you have set the warehouse location to 'home/jovyan/data/spark-warehouse'

if os.path.normpath("/home/jovyan/data/spark-warehouse") != warehouse_location:
    print('\x1b[6;37;41m' + 'Your path is not correct' + '\x1b[0m')
    

/home/jovyan/data/spark-warehouse
-Dderby.system.home=/home/jovyan/data/spark-warehouse


In [33]:



# Create the session
conf = (SparkConf()
    .set("spark.ui.port", "4041")
    .set('spark.jars', '/home/jovyan/scratch/postgresql-42.2.18.jar')
    .set("spark.sql.warehouse.dir", warehouse_location) # set above.
    .set("hive.metastore.schema.verification", False)
    .set("javax.jdo.option.ConnectionURL", "jdbc:derby:;databaseName=metastore_db;create=true") # settings to persist data for sparksql
    .set("javax.jdo.option.ConnectionDriverName", "org.apache.derby.jdbc.EmbeddedDriver") # settings to persist data for sparksql
    .set("javax.jdo.option.ConnectionUserName", 'userman') # may not need this
    .set("jdo.option.ConnectionPassword", "pwd") # may not need this
    .set("spark.driver.extraJavaOptions",java_options) # setting where the derby log files are created.
    .set("spark.sql.inMemoryColumnarStorage.compressed", True) # default
    .set("spark.sql.inMemoryColumnarStorage.batchSize",10000) # default
    )

# Create the Session (used to be context)
# you can move the number up or down depending on your memory and processors "local[*]" will use all.
# The conf from above is used in .config.
spark = SparkSession.builder \
    .master("local[7]") \
    .appName('test') \
    .config(conf=conf) \
    .enableHiveSupport() \
    .getOrCreate()



In [37]:
spark.sql('DROP DATABASE library CASCADE;')

DataFrame[]

In [38]:
# spark.stop()
# spark.sql('DROP DATABASE IF EXISTS irs990 CASCADE;')
# spark.sql("create database irs990")
spark.sql("SHOW DATABASES").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [39]:
 spark.sql("CREATE DATABASE library;")

DataFrame[]

In [46]:
#spark.sql("USE library")
#collection = spark.read.format("csv").load("../data/big/Library_Collection_Inventory.csv")
#print(collection.limit(5))
#collection.write.mode("overwrite").saveAsTable("collection")

spark.sql("USE library")
collection = spark.read.csv(path= "../data/big/Library_Collection_Inventory.csv", header = True)
print(collection.limit(5).show(5))
collection.write.mode("overwrite").saveAsTable("collection")

+-------+--------------------+--------------------+--------------------+---------------+--------------------+--------------------+--------+--------------+------------+------------+----------+---------+
| BibNum|               Title|              Author|                ISBN|PublicationYear|           Publisher|            Subjects|ItemType|ItemCollection|FloatingItem|ItemLocation|ReportDate|ItemCount|
+-------+--------------------+--------------------+--------------------+---------------+--------------------+--------------------+--------+--------------+------------+------------+----------+---------+
|2661617|Kindergarten diar...|  Portis, Antoinette|0061456918, 00614...|         [2010]|      HarperCollins,|Kindergarten Juve...|    jcbk|         ccpic|          NA|         cen|01/01/2020|        2|
| 321202|The sex life, bef...|Sadler, William S...|                null|          1938.|American Pub. Corp.,|Sex, Sexual healt...|    acbk|           cs6|          NA|         cen|01/01/2020| 

In [48]:
#spark.sql("USE library")
#checkouts = spark.read.format("csv").load("../data/big/Checkouts_by_Title.csv")
#print(checkouts.limit(5))
#checkouts.write.mode("overwrite").saveAsTable("checkouts")


spark.sql("USE library")
collection = spark.read.csv(path="../data/big/Checkouts_by_Title.csv",header=True)
print(collection.limit(5).show(5))
collection.write.mode("overwrite").saveAsTable("checkouts")

+----------+------------+------------+------------+-------------+---------+--------------------+--------------------+--------------------+--------------------+---------------+
|UsageClass|CheckoutType|MaterialType|CheckoutYear|CheckoutMonth|Checkouts|               Title|             Creator|            Subjects|           Publisher|PublicationYear|
+----------+------------+------------+------------+-------------+---------+--------------------+--------------------+--------------------+--------------------+---------------+
|   Digital|   OverDrive|       EBOOK|        2018|            4|        1|     The Storyteller|   Antonia Michaelis|Mystery, Young Ad...|              ABRAMS|           2013|
|   Digital|   OverDrive|   AUDIOBOOK|        2018|            4|       32|How Not to Be Wro...|    Jordan Ellenberg|Mathematics, Nonf...|       Books on Tape|           2014|
|  Physical|     Horizon|        BOOK|        2018|            4|       18|Rick Steves best ...| Steves, Rick, 1955-|Spa

In [49]:

spark.sql("SELECT * FROM collection LIMIT 5 ORDER").show()

+-------+--------------------+--------------------+--------------------+---------------+--------------------+--------------------+--------+--------------+------------+------------+----------+---------+
| BibNum|               Title|              Author|                ISBN|PublicationYear|           Publisher|            Subjects|ItemType|ItemCollection|FloatingItem|ItemLocation|ReportDate|ItemCount|
+-------+--------------------+--------------------+--------------------+---------------+--------------------+--------------------+--------+--------------+------------+------------+----------+---------+
|2940791|Baby Beatles! / C...|   Babypants, Caspar|                null|         [2013]|Aurora Elephant M...|Childrens songs J...|    jccd|          nccd|    Floating|         bea|12/01/2017|        1|
|2994551|Living simple, fr...|Frank, Cristin, 1...|1440325251, 97814...|         [2013]|Betterway Home Bo...|Home economics, H...|    acbk|          nanf|          NA|         rbe|12/01/2017| 

In [52]:
spark.sql("""
SELECT COLUMN_NAME
FROM collection.COLUMNS
WHERE TABLE_NAME = collection
""").show()

AnalysisException: Table or view not found: collection.COLUMNS; line 1 pos 24;
'Project ['COLUMN_NAME]
+- 'Filter ('TABLE_NAME = 'collection)
   +- 'UnresolvedRelation [collection, COLUMNS], [], false


In [1]:
dat = pd.read_parquet("../data/spark-warehouse/library.db/collection")

NameError: name 'pd' is not defined

In [64]:
#checkouts.scatter(x = Author,y = Checkouts)

#checkouts.plot(kind='scatter', x = 'Author', y = 'Publisher')

#plt.scatter(checkouts.Author, checkouts.Publisher)
# OR (with pandas 0.13 and up)
#df.plot(kind='scatter', x='col1', y='col2', s=df.col3)

NameError: name 'plt' is not defined